In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tcn import TCN
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import ParameterGrid

In [2]:
X_train = np.load('X_train.npy')  # Load your preprocessed training data
y_train = np.load('y_train.npy')
X_val = np.load('X_val.npy')  # Load your validation data
y_val = np.load('y_val.npy')
X_test = np.load('X_test.npy')  # Load your test data
y_test = np.load('y_test.npy')

In [3]:
num_classes = len(np.unique(y_train))

In [4]:
# Define the parameter grid
param_grid = {
    'batch_size': [16, 32],  # Limit batch size options
    'epochs': [5],  # Fewer epochs for quicker training
    'dropout_rate': [0.2, 0.3],  # Limited dropout rates
    'filters': [32, 64],  # Fewer filter options
    'kernel_size': [3, 5]  # Fewer kernel sizes
}

In [5]:
# Create a list of all parameter combinations
param_combinations = list(ParameterGrid(param_grid))
print(f"Total combinations to try: {len(param_combinations)}")

# Results storage
results = []

# Hyperparameter tuning with smaller fitting (limited epochs, fewer parameters)
for params in param_combinations:
    print(f"\nTraining with parameters: {params}")

    # Build the model with hyperparameters from the grid search
    model = Sequential()

    # Add TCN layers
    model.add(TCN(input_shape=(X_train.shape[1], X_train.shape[2]), nb_filters=params["filters"], kernel_size=params["kernel_size"]))
    model.add(Dropout(params["dropout_rate"]))  # Regularization
    model.add(Dense(32, activation='relu'))  # Smaller Dense layer
    model.add(Dense(num_classes, activation='softmax'))  # Output layer for classification

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model with the specified batch size and number of epochs
    model.fit(X_train, y_train, epochs=params["epochs"], batch_size=params["batch_size"], validation_data=(X_val, y_val))

    # Evaluate the model on the validation set
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy: {val_acc:.4f}")

    # Store the results
    results.append({
        "params": params,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

# Find the best combination based on validation accuracy
best_result = max(results, key=lambda x: x["val_acc"])
print("\nBest Parameters and Results:")
print(best_result)

Total combinations to try: 16

Training with parameters: {'batch_size': 16, 'dropout_rate': 0.2, 'epochs': 5, 'filters': 32, 'kernel_size': 3}


C:\Users\Senuli\Desktop\env\Lib\site-packages\tcn\tcn.py:227: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)



Epoch 1/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 19s 85ms/step - accuracy: 0.6499 - loss: 604.5148 - val_accuracy: 0.8780 - val_loss: 26.0801
Epoch 2/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.7637 - loss: 60.2846 - val_accuracy: 0.8626 - val_loss: 21.8519
Epoch 3/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 14s 85ms/step - accuracy: 0.7957 - loss: 23.6586 - val_accuracy: 0.8761 - val_loss: 13.5905
Epoch 4/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 15s 89ms/step - accuracy: 0.8021 - loss: 16.7837 - val_accuracy: 0.6830 - val_loss: 19.2605
Epoch 5/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 12s 74ms/step - accuracy: 0.7702 - loss: 15.7628 - val_accuracy: 0.8079 - val_loss: 14.8481
Validation Accuracy: 0.8079

Training with parameters: {'batch_size': 16, 'dropout_rate': 0.2, 'epochs': 5, 'filters': 32, 'kernel_size': 5}
Epoch 1/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step - accuracy: 0.6471 - loss: 389.9650 - val_accuracy: 0.8674 - val_loss: 20.0085
Epoch 2/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 15s 93ms/step - accuracy: 0.791

In [6]:
# Get the best parameters
best_params = best_result['params']

# Build the model with the best parameters
model = Sequential()
model.add(TCN(input_shape=(X_train.shape[1], X_train.shape[2]), nb_filters=best_params["filters"], kernel_size=best_params["kernel_size"]))
model.add(Dropout(best_params["dropout_rate"]))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model with the best parameters
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=best_params["epochs"], batch_size=best_params["batch_size"], validation_data=(X_val, y_val))

# Predict and evaluate on the test set
y_pred = np.argmax(model.predict(X_test), axis=-1)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Save the final model
model.save('tcn_best_model.keras')

Epoch 1/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 40s 222ms/step - accuracy: 0.7283 - loss: 246.9460 - val_accuracy: 0.8703 - val_loss: 17.9281
Epoch 2/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 36s 217ms/step - accuracy: 0.7960 - loss: 24.3337 - val_accuracy: 0.8309 - val_loss: 15.4625
Epoch 3/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 36s 218ms/step - accuracy: 0.8129 - loss: 13.2948 - val_accuracy: 0.8761 - val_loss: 9.9033
Epoch 4/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 35s 216ms/step - accuracy: 0.8201 - loss: 11.3407 - val_accuracy: 0.8511 - val_loss: 7.6497
Epoch 5/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 38s 234ms/step - accuracy: 0.8192 - loss: 11.7976 - val_accuracy: 0.8405 - val_loss: 6.1017
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 191ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.91      0.89       539
           1       0.82      0.77      0.79       298

    accuracy                           0.86       837
   macro avg       0.85      0.84      0.84       837
weight